In [1]:
import sys
import numpy as np
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.utils import plot_model
from keras import backend as K

import Generator as gen

Using TensorFlow backend.


In [2]:
learning_rate = 0.0001
batch_size = 1
img_height = 256
img_width = 256
img_channel = 1
data_path = 'data/dataset_256x256.npz'
model_path = 'model/'
steps_per_epoch=5
epochs=5
validation_step=2

In [16]:
class Multi_UNet():
    def __init__(self, learning_rate, batch_size, img_height, img_width, img_channel):
        self.lr = learning_rate
        self.bs = batch_size
        self.height = img_height
        self.width = img_width
        self.channel = img_channel
        self.model = self.create_model()
    
    def ConvBlock(self, x, num_filter, conv_kernel_size=(3,3), dropout=0.2,
                  pool_size=(2,2), hasPool=True):
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                      kernel_initializer='he_normal', padding='same') (x)
        conv = Dropout(dropout) (conv)
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                      kernel_initializer='he_normal', padding='same') (conv)
        if hasPool == True:
            pool = MaxPooling2D(pool_size)(conv)
            return conv, pool
        return conv
    
    def DeConvBlock(self, x, cat, num_filter=16, conv_kernel_size=(3,3), up_size=(2,2), 
                    up_stride=(2,2), dropout=0.2, axis=-1):
        deconv = Conv2DTranspose(num_filter, up_size, strides=up_stride, padding='same') (x)
        deconv = concatenate([deconv, cat], axis=axis)
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                      kernel_initializer='he_normal', padding='same') (deconv)
        conv = Dropout(dropout) (conv)
        conv = Conv2D(num_filter, conv_kernel_size, activation='elu', 
                    kernel_initializer='he_normal', padding='same') (conv)
        return conv
    
#     def mean_iou(y_true, y_pred):
#         print(y_pred)
#         prec = [[], []]    #segmentation, contour
#         for i in range(2):
#             for t in np.arange(0.5, 1.0, 0.05):
#                 y_pred_ = tf.to_int32(y_pred[i] > t)
#                 score, up_opt = tf.metrics.mean_iou(y_true[i], y_pred_, 2, y_true[i])
#                 K.get_session().run(tf.local_variables_initializer())
#                 with tf.control_dependencies([up_opt]):
#                     score = tf.identity(score)
#                 prec[i].append(score)
#         return K.mean(K.stack(prec[0]), axis=0), K.mean(K.stack(prec[1]), axis=0)

    def mean_iou(y_true, y_pred):
        print(y_pred)
        prec = []    #segmentation, contour
        for i in range(2):
            for t in np.arange(0.5, 1.0, 0.05):
                y_pred_ = tf.to_int32(y_pred > t)
                score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2, y_true)
                K.get_session().run(tf.local_variables_initializer())
                with tf.control_dependencies([up_opt]):
                    score = tf.identity(score)
                prec.append(score)
        return K.mean(K.stack(prec), axis=0)
    
    
    def create_model(self):
        inputs = Input((self.height, self.width, self.channel))
        
        # down-sampling
        conv1, pool1 = self.ConvBlock(inputs, 16, (3,3), 0.1, (2,2))
        conv2, pool2 = self.ConvBlock(pool1, 32, (3,3), 0.1, (2,2))
        conv3, pool3 = self.ConvBlock(pool2, 64, (3,3), 0.2, (2,2))
        conv4, pool4 = self.ConvBlock(pool3, 128, (3,3), 0.2, (2,2))
        conv5 = self.ConvBlock(pool4, 256, (3,3), 0.3, hasPool=False)
        
        # up-sampling
        conv6_s = self.DeConvBlock(conv5, conv4, 128, (3,3), (2,2), (2,2), 0.2)
        conv7_s = self.DeConvBlock(conv6_s, conv3, 64, (3,3), (2,2), (2,2), 0.2)
        conv8_s = self.DeConvBlock(conv7_s, conv2, 32, (3,3), (2,2), (2,2), 0.1)
        conv9_s = self.DeConvBlock(conv8_s, conv1, 16, (3,3), (2,2), (2,2), 0.1, axis=3)
        outputs_s = Conv2D(1, (1, 1), activation='sigmoid') (conv9_s)
        
        conv6_c = self.DeConvBlock(conv5, conv4, 128, (3,3), (2,2), (2,2), 0.2)
        conv7_c = self.DeConvBlock(conv6_c, conv3, 64, (3,3), (2,2), (2,2), 0.2)
        conv8_c = self.DeConvBlock(conv7_c, conv2, 32, (3,3), (2,2), (2,2), 0.1)
        conv9_c = self.DeConvBlock(conv8_c, conv1, 16, (3,3), (2,2), (2,2), 0.1, axis=3)
        outputs_c = Conv2D(1, (1, 1), activation='sigmoid') (conv9_c)
        
        model = Model(inputs=[inputs], outputs=[outputs_s, outputs_c])
        adam = optimizers.Adam(lr = self.lr)
        model.compile(optimizer=adam, loss='binary_crossentropy', loss_weights=[1.,1.],
                      metric=[mean_iou])
        print(model.summary())
        
        return model


In [17]:
def main(args):
    # define model
    multi_unet = Multi_UNet(learning_rate, batch_size, img_height, img_width, img_channel)
    model = multi_unet.model
#     plot_model(model, to_file='model.png')
    print("generate model!")
    
    # define generator
    trainGenerator = gen.SegDataGenerator(validation_split=0.2)
    train_data = trainGenerator.flow_from_directory(data_path, subset='training', batch_size=batch_size,
                                                   class_mode='segmentation', color_mode='grayscale')
    val_data = trainGenerator.flow_from_directory(data_path, subset='validation')
    print("data generator!")
    
    # training
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    checkpoint = ModelCheckpoint(model_path+'weight.{epoch:02d}.hdf5', monitor='val_loss',
                                 mode='min', period=1)
    model.fit_generator(train_data, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                        callbacks=[checkpoint], validation_data=val_data, 
                        validation_steps=validation_step, shuffle=True)
    
    

In [18]:
if __name__ == '__main__':
    main(sys.argv)

[<tf.Tensor 'conv2d_119/Sigmoid:0' shape=(?, 256, 256, 1) dtype=float32>, <tf.Tensor 'conv2d_128/Sigmoid:0' shape=(?, 256, 256, 1) dtype=float32>]
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 256, 256, 1)   0                                            
____________________________________________________________________________________________________
conv2d_101 (Conv2D)              (None, 256, 256, 16)  160         input_5[0][0]                    
____________________________________________________________________________________________________
dropout_49 (Dropout)             (None, 256, 256, 16)  0           conv2d_101[0][0]                 
____________________________________________________________________________________________________
conv2d_102 (Conv2D)              (None, 256, 